## 9.7 序列号序列的学习(seq2seq)
本节，我们将使用两个循环神经网络的编码器和解码器，并将其应用于序列到序列(sequence to sequence, seq2seq)类的学习任务。

遵循编码器-解码器架构的设计原则，循环神经网络编码器使用长度可变的序列作为输入，将其转换为固定形状的隐状态。换言之，输入序列的信息被编码到循环神经网络编码器的隐状态中。为了连续生产输出序列的词元，独立的循环神经网络解码器是基于输入序列的编码器信息和输出序列已经看见的或者生成的词元来预测下一个词元。图9.7.1演示了如何在机器翻译中使用两个RNN进行序列到序学习。

<div align=center>
<img src='../../pics/9_7_1.jpeg' width='50%'>
</div>

在图9.7.1中，特定的“<eos>”表示序列结束词元。一旦输出序列生成此词元，模型就会停止预测。在循环神经网络解码器的初始化时间步，有两个特定的设计决定：首先，特定的“<bos>”表示序列开始词元，它是解码器的输入序列的第一个词元。其次，使用RNN编码器最终的编码信息送入到解码器中来生成输出序列。在一些其他设计中，如图9.7.1所示，编码器最终的隐状态在每一个时间步都作为解码器的输入序列的一部分。类似于8.3节中的语言模型训练，可以允许标签成为原始的输出序列，从源序列词元“<bos>”、“Ils”、“regardent”、“.”到新序列词元“Ils”、“regardent”、“.”、“<eos>”来移动预测到位置。

下面，我们动手构建图9.7.1的设计，并将基于9.5节中介绍的“英-法”数据集来训练这个机器翻译模型

In [1]:
import collections
import math
import torch
from torch import nn
from d2l import torch as d2l

### 9.7.1 编码器
从技术上讲，编码器将长度可变的输入序列转换成形状固定的上下文变量$c$，并且将输入序列的信息在该上下文变量中进行编码。如图9.7.1所示，可以使用RNN来设计编码器

考虑由一个序列组成的样本(批量大小是1)。假设输入序列是$x_1, ..., x_T$，其中$x_t$是输入文本序列中的第t个词元。在时间步t，RNN将词元$x_t$的输入特征向量$x_t$和$h_{t-1}$(即上一时间步的隐状态)转换为$h_t$(即当前步的隐状态)。使用一个函数$f$来描述RNN的循环层所做的变换：
$$
h_t = f(x_t, h_{t-1})
\tag{9.7.1}
$$
总之，编码器通过选定的函数q，将所有时间步的隐状态转换为上下文变量：
$$
c = q(h_1, ..., h_T)
\tag{9.7.2}
$$

比如，当选择$q(h_1, ..., h_T) = h_T$时，上下文变量仅仅是输入序列在最后时间步的隐状态$h_T$。

到目前为止，我们使用的是一个单向循环神经网络来设计编码器，其中隐状态只依赖于输入子序列，这个子序列是由输入序列的开始位置到隐状态所在的时间步的为止(包括隐状态所在的时间步)组成。我们也可以使用双向循环神经网络构造编码器，其中隐状态依赖于两个输入子序列，两个子序列是由隐状态所在的时间步的位置之前的序列和之后的序列(包括隐状态所在的时间步)，因此隐状态对整个序列的信息都进行了编码。

现在，让我们实现循环神经网络编码器。注意，我们使用了嵌入层(embedding layer)来获得输入序列中每个词元的特征向量。嵌入层的权重是一个矩阵，其行数等于输入词表的大小(vocab_size)，其列数等于特征向量的维度(embed_size)。对于任意输入词元的索引$i$，嵌入层获取权重矩阵的第i行(从0开始)以返回其特征向量。

In [4]:
#@save
class Seq2SeqEncoder(d2l.Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)

    def forward(self, X, *args):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

下面，我们实例化上述编码器的实现：我们使用一个两层门控循环单元编码器，其隐藏单元数为16.给定一小批量的输入序列X(批量大小为4， 时间步为7)。在完成所有时间步后，最后一层的隐状态的输出是一个张量(output由编码器的循环层返回)，其形状为(时间步数, 批量大小, 隐藏单元数)。

In [5]:
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval()
X = torch.zeros((4, 7), dtype=torch.long)
output, state = encoder(X)
output.shape

torch.Size([7, 4, 16])

由于这里使用的是门控循环单元，所以在最后一个时间步的多层隐状态的形状是(隐藏层的数量, 批量大小, 隐藏单元的数量)。如果使用长短期记忆网络，state中还将包含记忆单元信息。

In [6]:
state.shape

torch.Size([2, 4, 16])

### 9.7.2 解码器
正如上文提到的，编码器输出的上下文变量$c$对整个输入序列$x_1, ..., x_T$进行编码。来自训练数据集的输出序列$y_1, y_2, ..., y_T^{'}$，对于每个时间步$t'$(与输入序列或编码器的时间步t不同)，解码器输出$y_t'$的概率取决于先前的输出子序列$y_1, ..., y_{t' -1}$和上下文变量$c$，即$P(y_{t'} \mid y_1, ..., y_{t' -1}, c)$。

为了在序列上模型化这种条件概率，我们可以使用另一个循环神经网络作为解码器。在输出序列上的任意时间步$t'$，循环神经网络将来自上一时间步的输出$y_{t' -1}$和上下文变量$c$作为其输入，然后在当前时间步将它们和上一隐状态$s_{t' -1}$转换为隐状态$s_{t'}$。因此，可以使用函数$g$来表示解码器的隐藏层的变换：
$$
s_{t'} = g (y_{t' - 1}, c, s_{t' -1})
\tag{9.7.3}
$$

在获得解码器的隐状态之后，我们可以使用输出层和softmax操作来计算在时间步$t'$时输出$y_{t'}$的条件概率分布$P(y_{t'} \mid y_1, ..., y_{t' -1}, c)$。

根据图9.7.1，当实现解码器时，我们直接使用编码器最后一个时间步来初始化解码器的隐状态。这就要求使用循环神经网络实现的编码器和解码器具有相同数量的层和隐藏单元。为了进一步包含经过编码的输入序列的信息，上下文变量在所有的时间步与解码器的输入进行拼接(concatenate)。为了预测输出词元的概率分布，在循环神经网络解码器的最后一层使用全连接层来变换隐状态。

In [ ]:
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, 
                dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                    dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # 输出‘X’的形状: (batch_size, num_steps, embed_size)
        X = self.embedding(X).premute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        